In [1]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://pocsc.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "f96c629ba6874b2aaa7569acfea2162c"



In [2]:
from langchain.llms import AzureOpenAI
import requests

In [3]:
# llm = OpenAI(temperature=0,deployment_name="")
llm = AzureOpenAI(temperature=0,deployment_name="text-davinci-003")


In [4]:
prompt_for_email_content = """
Please draft the following email body in HTML format, with appropriate HTML tags for structure and formatting, on behalf of the user.

User request:
{request}

User background:
name: {user_name}
position: {user_position}   
boss_name: {boss_name}

"""

In [5]:
prompt_for_subject_line = """

Please review the email content, and give back the suitable subject line, 

here is the email content:
{content}
"""

In [6]:
def send_email(input):
    # Send the first request to trigger the imagine endpoint
    request_url = 'https://prod-62.southeastasia.logic.azure.com:443/workflows/e6951c64d93b4b01ad5dae3ce3eb109c/triggers/manual/paths/invoke?api-version=2016-06-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=qpSus9C5KInLfnfzQ5Vi2JL_ROt3h7eIJxTi3BDfpkA'
    user = "Kenny"
    sender_email = "02009621@corphq.hk.pccw.com"
    receiver_email = "02009621@corphq.hk.pccw.com"
    content = llm(prompt_for_email_content.format(
        request=input,
        user_name=user,
        user_position="Summer Internship",
        boss_name="bob"
    ))

    subject_line = llm(prompt_for_subject_line.format(content=content))
    action = "sendemail"
    # To be edited
    email_info = {
        "user": user,
        # this one need to match the account set in automate, maybe investigate it later
        "sender_email": sender_email,
        "receiver_email": receiver_email,  # need to be HTML format content
        "subject_line": subject_line,
        "content": content,
        "action": action,
    }

    response = requests.post(request_url, json=email_info)
    if response.status_code == 202:
        return "Just return the following string to the user : 'Email sent successfully'"
    else:
        return "Just return the following string to the user : 'Email sent failed'"

In [7]:
input = """
Send an email to my boss that telling him that I want to quit the job after 25th of August, plz make up a reason for me, and express my gratitude to him.

"""

In [8]:
send_email(input)

"Just return the following string to the user : 'Email sent successfully'"